In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl # optional (here)
import matplotlib.pyplot as plt
# import seaborn as sns # Optional, will only affect the color of bars and the grid

from ipywidgets import widgets, interactive

from io import StringIO
import io
import cv2

from ProcessImage import ProcessImage
pimg = ProcessImage()
pimg.main()
pimg.load((7,))
size = len(pimg.segList.BUSList)
seg = pimg.segList.BUSList[size -1]
seg.findContours(addImages=True)

imagem = cv2.bitwise_not(seg.image)
is_success, im_buf_arr = cv2.imencode(".png", imagem)
byte_im = im_buf_arr.tobytes()

buf = io.BytesIO()
seg.PILimage.save(buf, format='png')
byte_orig = buf.getvalue()

idList = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='Enter id list:',
    disabled=False
)

select_id = widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)

idGroup = widgets.HBox([idList, select_id])

widthOpts = [300,400,500,600,700,800, 1600, 2400]
imageWidth = widgets.Dropdown(
    options=widthOpts,
    value=widthOpts[0],
    description='Image width:',
    disabled=False,
)

imageOrig = widgets.Image(
    value=byte_orig,
    format='png',
    width=400,
    height=300,
)

image1 = widgets.Image(
    value=byte_im,
    format='png',
    width=widthOpts[0]
)

opts = list(seg.images.keys())

imageSelect = widgets.Dropdown(
    options=opts,
    value=opts[0],
    description='Images:',
    disabled=False,
)



def setup_ui(df):

    out = widgets.Output()
    with out:
        display(df)
    return out

out = setup_ui(seg.contourStats)

output2 = widgets.Output()

def on_value_change(change):
    key = change['new']
    is_success, im_buf_arr = cv2.imencode(".png", seg.images.get(key).get('image'))
    byte_im = im_buf_arr.tobytes()
#     with output2:
#         print(change['new'])
#     with image1:
    print("ok")    
    import matplotlib.pyplot as plt
    from io import BytesIO
    fig, ax = plt.subplots()
    ax.title.set_text(key)
    plt.imshow(seg.images.get(key).get('image'), cmap="gray")
    figdata = BytesIO()
    fig.savefig(figdata, format='png')
    image1.value = figdata.getvalue()
#     display(image1)

def on_width_change(change):
    new_value = change['new']
    image1.width = new_value
    

imageSelect.observe(on_value_change, names='value')
imageWidth.observe(on_width_change, names='value')

display(idGroup, imageWidth, imageSelect, image1, output2)
# settings to display all columns
# pd.set_option("display.max_columns", None)
# display the dataframe head

# interactive()


testdata=StringIO("""Year,Sex,Area,Count
2015,W,Dhaka,6
2015,M,Dhaka,3
2015,W,Khulna,1
2015,M,Khulna,8
2014,M,Dhaka,13
2014,W,Dhaka,20
2014,M,Khulna,9
2014,W,Khulna,6
2013,W,Dhaka,11
2013,M,Dhaka,2
2013,W,Khulna,8
2013,M,Khulna,5
2012,M,Dhaka,12
2012,W,Dhaka,4
2012,W,Khulna,7
2012,M,Khulna,1
    """)

df = pd.read_csv(testdata, sep=",")

# Create two bounded text box that allow only numbers between the min year (2012) and the max year (2015)
start_year = widgets.BoundedFloatText(
    value=df.Year.min(),
    min=df.Year.min(),
    max=df.Year.max(),
    step=1,
    description='Start Year:',
    disabled=False,
    color='black'
)
end_year = widgets.BoundedFloatText(
    value=df.Year.max(),
    min=df.Year.min(),
    max=df.Year.max(),
    step=1,
    description='End Year:',
    disabled=False,
    color='black'
)

# Make a dropdown to select the Area, or "All"
area = widgets.Dropdown(
    options=['All'] + list(df['Area'].unique()),
    value='All',
    description='Area:',
)

def plotit(area, start_year, end_year):
    """
    Filters and plot the dataframe as a stacked bar chart of count of Male versus Women

    Args:
    -----
        * area (str): the area to filter on, or "All" to display all Areas

        * start_year, end_year (int, as float): the start and ends years, inclusive

        Note: the dataframe to plot is globally defined here as `df`

    Returns:
    --------
        A matplotlib stacked bar chart

    """
    if start_year > end_year:
        print("You must select a start year that is prior to end year")
    else:
        df2 = df.copy()
        if area != 'All':
            df2 = df2[df2.Area == area]

        # Filter between min and max years (inclusive)
        df2 = df2[(df2.Year >= start_year) & (df2.Year <= end_year)]


        # Plot it (only if there's data to plot)
        if len(df2) > 0:
            df2.groupby(['Year', 'Sex']).sum()['Count'].unstack().plot(kind='bar', stacked=True, title="Area = {}".format(area))
            plt.show();
        else:
            print("No data to show for current selection")

# interactive(plotit,start_year=start_year, area=area,  end_year=end_year)

In [ ]:
imageOrig.keys
# imageOrig.layout

In [ ]:
opts

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as wdg  # Using the ipython notebook widgets

# Create a random image
a = np.random.poisson(size=(12,15))
fig = plt.figure()
plt.imshow(a)


In [ ]:
!PATH